In [1]:
import os
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

In [2]:
AZURE_OPENAI_KEY = "4f8e9d40be39474c96fa1327dbd47516"
AZURE_OPENAI_ENDPOINT = "https://openai-resource-team-3-france.openai.azure.com/"
OPENAI_API_VERSION = "2023-03-15-preview"

WOLFRAM_ALPHA_APPID = "3HGHX2-433LKG3JK7"

os.environ["OPENAI_API_KEY"] = AZURE_OPENAI_KEY
os.environ["OPENAI_API_BASE"] = AZURE_OPENAI_ENDPOINT
os.environ["OPENAI_API_VERSION"] = OPENAI_API_VERSION
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["WOLFRAM_ALPHA_APPID"] = WOLFRAM_ALPHA_APPID

In [3]:
math_llm = AzureChatOpenAI(deployment_name="gpt35-team-3-0301", max_tokens=100, temperature=0.2)
summary_llm = AzureChatOpenAI(deployment_name="gpt35-team-3-0301")
# text = "How to be happy?"
# print(chatgpt([HumanMessage(content=text)]).content)

In [4]:
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory, CombinedMemory, ConversationSummaryMemory

from langchain.agents import AgentType, Tool, initialize_agent
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain.utilities.wolfram_alpha import WolframAlphaAPIWrapper

In [5]:
# Trying to create a base pre-prompt for math
context = """\
The following text is simply an guide that the AI must follow after receiving this text input. \
This prompt only exists to give context to the AI. \
This is a friendly conversation between a human and an AI. \
The AI serves the purpose of being a mathematics tutor. \
The human is considered to be a highschooler learning in the Portuguese schools, which follow the portuguese teaching system. \
The AI should help the human by giving clear detail on how to solve an exercise or how to understand a concept. \
The AI must not use techniques that are considered to be of higher educational experience. \
If the AI is given a major concept, it should try to explain it to the human without asking if tools are needed. The AI should make that decision by itself. \
Any equations provided by the AI should be written delimited by ```. \
If the AI does not know the answer to a question, it truthfully says it does not know. \
"""

In [6]:
wolframalpha = WolframAlphaAPIWrapper()
tools = [
    Tool(
        name="Math Helper",
        func=wolframalpha.run,
        description="useful for when you need to answer questions math related"
    )
]

In [7]:
#memory = ConversationBufferMemory(memory_key="chat_history")

conv_memory = ConversationBufferWindowMemory(
    memory_key="chat_history",
    input_key="input",
    k=1
)

summary_memory = ConversationSummaryMemory(llm=summary_llm, input_key="input")
# Combined
memory = CombinedMemory(memories=[conv_memory, summary_memory])

In [8]:
agent_chain = initialize_agent(
    tools, math_llm, agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory
)

In [10]:
agent_chain.run(input=context)



> Entering new AgentExecutor chain...


OutputParserException: Could not parse LLM output: `Hello! How can I assist you with your math studies today?`

In [ ]:
agent_chain.run(input="Triangular similarity")

In [ ]:
agent_chain.run(input="How did you get to that answer?")

In [ ]:
agent_chain.run(input="Can you give me the second derivative?")